In [22]:
import pandas as pd
import pyam 
from nomenclature.codelist import MetaCodeList
from pathlib import Path

In [23]:
def _check_if_values_allowed(df: pyam.IamDataFrame, meta_indicator, meta_code_list: MetaCodeList) -> pyam.IamDataFrame:
    not_allowed = []
    for value in df.meta[meta_indicator].values:
        if value not in meta_code_list.mapping[meta_indicator].allowed_values:
            not_allowed.append(value)
    if not_allowed:
        raise ValueError(f"{not_allowed} indicator value(s) in the {meta_indicator} column are not allowed")
    return df

In [24]:
def validate_meta_indicators(df: pyam.IamDataFrame, path: Path("**/*")) -> pyam.IamDataFrame:
    meta_code_list = MetaCodeList.from_directory(name='Meta', path=path)
    for meta_indicator in df.meta.columns:
        if meta_indicator in meta_code_list.mapping:
            _check_if_values_allowed(df=df, meta_indicator=meta_indicator, meta_code_list=meta_code_list)
            return df
    raise ValueError(f"There are no recognized meta_indicators (besides exclude) in IamDataFrame")

In [25]:
TEST_YEARS = [2005, 2010]
DF = pd.DataFrame(
    [
        ["model_a", "scen_a", "World", "Primary Energy", "EJ/yr", 1, 6.0],
        ["model_a", "scen_a", "World", "Primary Energy|Coal", "EJ/yr", 0.5, 3],
        ["model_a", "scen_b", "World", "Primary Energy", "EJ/yr", 2, 7],
    ],
    columns=pyam.IAMC_IDX + TEST_YEARS,
)
path= Path("tests/data/definitions/meta")


df=pyam.IamDataFrame(DF)
validate_meta_indicators(df=df, path=path)

<class 'pyam.core.IamDataFrame'>
Index:
 * model    : model_a (1)
 * scenario : scen_a, scen_b (2)
Timeseries data coordinates:
   region   : World (1)
   variable : Primary Energy, Primary Energy|Coal (2)
   unit     : EJ/yr (1)
   year     : 2005, 2010 (2)
Meta indicators:
   exclude (bool) False (1)